In [3]:
import camelot
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import pprint
import json
import re

### Occurrences

In [107]:
tables2 = camelot.read_pdf('Regions.pdf', pages='59-83, 92', flavor='stream')
print(tables2)
print (tables2[0].parsing_report)

tables3 = camelot.read_pdf('Regions.pdf', pages='84-91', flavor='stream')
print(tables3)
print (tables3[0].parsing_report)

tables4 = camelot.read_pdf('Regions.pdf', pages='93-137', flavor='stream')
print(tables4)
print (tables4[0].parsing_report)

<TableList n=27>
{'accuracy': 100.0, 'whitespace': 61.33, 'order': 1, 'page': 59}


In [271]:
df_i = pd.DataFrame()
df_j = pd.DataFrame()
df_k = pd.DataFrame()
df_l = pd.DataFrame()

for j in range(27):
    df_j = pd.concat([df_j, tables2[j].df])
df_j.replace('', np.NaN, inplace=True)
df_j.fillna(method='ffill', inplace=True)
df_j['a'] = df_j[0] + ' | ' + df_j[1] + ' | ' + df_j[2]
df_j['b'] = df_j[4] + ' | ' + df_j[5] + ' | ' + df_j[6] + ' | ' + df_j[7]
df_j.drop(columns=[0,1,2,3,4,5,6,7], inplace=True)
df_j = df_j.apply(lambda col: col.str.lower())

for k in range(8):
    df_k = pd.concat([df_k, tables3[k].df])
df_k.replace('', np.NaN, inplace=True)
df_k.fillna(method='ffill', inplace=True)
df_k['a'] = df_k[1] + ' | ' + df_k[2] + ' | ' + df_k[3]
df_k['b'] = df_k[4] + ' | ' + df_k[5]
df_k.drop(columns=[0,1,2,3,4,5], inplace=True)
df_k = df_k.apply(lambda col: col.str.lower())
    
for l in range(45):
    df_l = pd.concat([df_l, tables4[l].df])
df_l.replace('', np.NaN, inplace=True)
df_l.fillna(method='ffill', inplace=True)
df_l['a'] = df_l[1] + ' | ' + df_l[2] + ' | ' + df_l[3]
df_l['b'] = df_l[4] + ' | ' + df_l[5]
df_l.drop(columns=[0,1,2,3,4,5], inplace=True)
df_l = df_l.apply(lambda col: col.str.lower())

In [287]:
complete = pd.DataFrame()
complete = pd.concat([complete, df_j, df_k, df_l])
complete

,a,b
0,common name | scientific | vernacular,NaN
1,common name | name | name used,NaN
2,black kite | milvus migrans | name used,NaN
3,brahminy kite | haliastur indus | lawin (tag.)...,NaN
4,brahminy kite | haliastur indus | banog (bis.)...,NaN
...,...,...
39,phocoenidae | finless porpoise | neophocaena p...,"sighted in the south china sea, northwest of |..."
40,phocoenidae | finless porpoise | neophocaena p...,palawan | and eastern sulu sea
41,dugongidae | dugong | dugong dugon,"m\nthroughout the philippine archipelago, indi..."
42,dugongidae | dugong | dugong dugon,"pacific oceans, culion | and eastern sulu sea"


In [289]:
b = complete[(complete['b'].str.contains('distribution') == True)].index
b = b.tolist()
a = complete[(complete['a'].str.contains('common name') == True) | (complete['a'].str.contains('source') == True) ].index
a = a.tolist()

ulist = a + b

complete.drop(index=ulist, inplace=True)
complete.reset_index(drop=True, inplace=True)

In [638]:
host_list = ['Sus scrofa', 'Gallus gallus', 'Columbidae',
       'Canis lupus familiaris', 'Bubalus bubalis', 'Feliformia',
       'Felis catus', 'Oreochromis', 'Aedes aegypti',
       'Macaca fascicularis', 'Armigeres', 'Bos taurus', 'Sus sp.',
       'Eonycteris spelaea', 'Rousettus amplexicaudatus', 'Suidae',
       'Sus scrofa domesticus', 'Equus caballus', 'Trachinotus blochii',
       'Culex bitaeniorhynchus', 'Penaeus monodon','Sus','Canis','Bubalus',
        'Felis','Aedes','Bos','Equus','Trachinotus','Culex','Penaeus']

host_list = [i.lower() for i in host_list]

In [142]:
def is_word_in_text(word, text):
    pattern = r'(^|[^\w]){}([^\w]|$)'.format(word)
    pattern = re.compile(pattern, re.IGNORECASE)
    matches = re.search(pattern, text)
    return bool(matches)

if __name__ == '__main__':
    import doctest
    doctest.testmod()

In [639]:
avail_wwf = pd.DataFrame(data={'a':[''],'b':['']})

for i in host_list:
    for j in range(len(complete)):
        if is_word_in_text(i, str(complete['a'].iloc[j])) == True:
            avail_wwf = avail_wwf.append(complete.iloc[j])

In [643]:
def reverseGeocode(coordinates):
    result = rg.search(coordinates)
    res = result[0]['admin1']
    return(res)

In [641]:
cols=['dwc:county','dwc:genus', 'idigbio:geoPoint',
       'dwc:locality','dwc:scientificName']

dig = pd.read_csv('./idig/occurrence.csv', usecols=cols)
dig.rename({'dwc:county':'county','dwc:genus':'genus', 'idigbio:geoPoint':'geopoint',
       'dwc:locality':'locality','dwc:scientificName':'sci_name'}, axis=1, inplace=True)

In [644]:
for i in range(len(dig)):
    if type(dig['geopoint'].iloc[i]) == str:
        cords = []
        test_string = dig['geopoint'].iloc[i]
        res = json.loads(test_string)
        cords.append(res['lat'])
        cords.append(res['lon'])
        dig['geopoint'].iloc[i] = reverseGeocode(cords)

In [647]:
dig.fillna('empty', inplace=True)
dig['a'] = dig['genus'] + ' | ' + dig['sci_name']
dig['b'] = dig['county'] + ' | ' + dig['locality'] + ' | ' + dig['geopoint'] 
dig.drop(columns=['county','genus','geopoint','locality','sci_name'], inplace=True)
dig = dig.apply(lambda col: col.str.lower())
dig

,a,b
0,trachinotus | trachinotus blochii,empty | pandanan island; between cebu and boho...
1,penaeus | penaeus monodon,"empty | negros island, tanon strait, binoy | e..."
2,trachinotus | trachinotus blochii,empty | port caltom; busuanga island | empty
3,trachinotus | trachinotus blochii,"empty | philippines, cebu mkt. | empty"
4,penaeus | penaeus monodon,"empty | luzon island, manila | empty"
...,...,...
86,trachinotus | trachinotus blochii,"empty | abuyog, (leyte); between samar and ley..."
87,macaca | macaca fascicularis,empty | mt. pangasugan | eastern visayas
88,macaca | macaca fascicularis,empty | maripipi | empty
89,eonycteris | eonycteris spelaea,"empty | gigmoto, 1 km n, 8.5 km w | bicol"


In [648]:
occs = pd.concat([dig, avail_wwf], ignore_index=True)
occs

,a,b
0,trachinotus | trachinotus blochii,empty | pandanan island; between cebu and boho...
1,penaeus | penaeus monodon,"empty | negros island, tanon strait, binoy | e..."
2,trachinotus | trachinotus blochii,empty | port caltom; busuanga island | empty
3,trachinotus | trachinotus blochii,"empty | philippines, cebu mkt. | empty"
4,penaeus | penaeus monodon,"empty | luzon island, manila | empty"
...,...,...
144,c. suidae | palawan bearded pig | sus barbatus...,vu | calauit; culion; coron; and adjacent isla...
145,suidae | bearded pig | sus barbatus,"r | bugsuc, calauit, culion, coron, and adjacent"
146,suidae | visayan warty pig | sus cebifrons,e | negros-panay faunal region only
147,suidae | philippine warty pig | sus philippensis,"e | mindoro, basilan, samar and camiguin"


In [847]:
provs = pd.read_csv('./region/provs.txt', delimiter=':')
provs['Prov'] = provs['Prov'].str.lower()
provs['Hosts'] = ''

In [848]:
def mark_regions(occ_i):
    oc_list = re.split(', | \s | \| | , |\s | \| , |\s', occs['b'].iloc[occ_i])
    for i in range(len(provs)):
        List1 = re.split(', | \s | \| | , |\s | \| , |\s', provs['Prov'].iloc[i])
        check =  any(item in oc_list for item in List1)
 
        if check is True:
            provs.at[i,'Hosts'] += '|'+ occs['a'].iloc[occ_i]
        else :
            continue
            
for i in range(len(occs)):
    mark_regions(i)

In [849]:
hosts = provs[['Regs','Hosts']]

### Mapping!!

In [836]:
viruses = pd.read_csv('clusters.csv')
viruses['Host'] = viruses['Host'].str.lower()
viruses['counts'] = 0
for i in range(len(viruses)):
    if 'homo sapiens' not in viruses['Host'].iloc[i]:
        viruses['Host'].iloc[i] += ',homo sapiens'

C:\Users\hbalcera\Miniconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [837]:
for i in range(len(viruses)):
    if ',' in viruses['Host'].iloc[i]:
        numCommas = viruses['Host'].iloc[i].count(',')
        viruses['counts'].iloc[i] = numCommas +1
    else:
        if viruses['Host'].iloc[i].isprintable() == True:
            viruses['counts'].iloc[i] = 1

vhost = viruses.groupby(['Cluster_Labels'])['Host'].apply(','.join)
vhost = pd.DataFrame(vhost)

In [838]:
def uniquify(string):
    output = []
    seen = set()
    for word in string.split(','):
        if word not in seen:
            output.append(word)
            seen.add(word)
    return ','.join(output)

for index, val in enumerate(vhost['Host']):
    value = uniquify(val)
    vhost.at[index, 'Host'] = value

In [843]:
def mark_regions(occ_i):
    oc_list = re.split(', | \s | \| | , |\s | \| , |\s', occs['b'].iloc[occ_i])
    for i in range(len(provs)):
        List1 = re.split(', | \s | \| | , |\s | \| , |\s', provs['Prov'].iloc[0])
        check =  any(item in oc_list for item in List1)
 
        if check is True:
            provs.at[i,'Hosts'] += '|'+ occs['a'].iloc[occ_i]
        else :
            continue
            
for i in range(len(occs)):
    mark_regions(i)

,Regs,Hosts
0,NCR,|penaeus | penaeus monodon|eonycteris | eonyct...
1,CAR,|penaeus | penaeus monodon|eonycteris | eonyct...
2,Region I,|penaeus | penaeus monodon|eonycteris | eonyct...
3,Region II,|penaeus | penaeus monodon|eonycteris | eonyct...
4,Region III,|penaeus | penaeus monodon|eonycteris | eonyct...
5,Region IVA,|penaeus | penaeus monodon|eonycteris | eonyct...
6,Region IVB,|penaeus | penaeus monodon|eonycteris | eonyct...
7,Region V,|penaeus | penaeus monodon|eonycteris | eonyct...
8,Region VI,|penaeus | penaeus monodon|eonycteris | eonyct...
9,Region VII,|penaeus | penaeus monodon|eonycteris | eonyct...


In [946]:
hosts['Counts'] = ''

def chost_regs(occ_i):
    oc_list = vhost['Host'].iloc[occ_i].split(',')
    for i in range(len(hosts['Hosts'])):
        List1 = re.sub(r'^\|{1}','',hosts['Hosts'].iloc[i])
        List1 = List1.replace(' | ',',').replace('|',',').replace('. ',',')
        List1 = List1.split(',')
        check =  any(item in oc_list for item in List1)
        matches = list((item in oc_list for item in List1))
        items = len(matches)
 
        if check is True:
            hosts.at[i,'Counts'] = items
        else :
            continue
            
for i in range(len(vhost)):
    chost_regs(i)

In [949]:
final_host = hosts[['Regs','Counts']]

### Region Demographics

In [502]:
table = camelot.read_pdf('./region/healthcare.pdf', pages='5', flavor='stream')
hc = table[0].df
hc.rename({0:'AREA', 1:'Doctors', 2:'Nurses', 3:'Medtech',4:'Beds'}, axis=1,inplace=True)
hc.at[22,'AREA'] = 'ARMM - Bangsamoro Autonomous Region in Muslim Mindanao'
hc.at[5,'AREA'] = 'NCR - National Capital Region'
hc.at[6,'AREA'] = 'CAR - Cordillera Administrative Region'
hc.at[10,'AREA'] = 'Region IVA - CALABARZON'
hc.at[11,'AREA'] = 'Region IVB - MIMAROPA'
hc.drop([0,1,2,3,4,21,23],axis=0, inplace=True)
hc['AREA'] = hc['AREA'].str.replace("\u002D","-")
hc['AREA'] = hc['AREA'].str.replace("\u2013","-")
hc['AREA'] = hc['AREA'].str.partition('-')[0].str.upper()
hc.reset_index(inplace=True)
hc.drop(columns='index', inplace=True)
hc

<TableList n=1>
{'accuracy': 99.25, 'whitespace': 18.33, 'order': 1, 'page': 5}


,AREA,Doctors,Nurses,Medtech,Beds
0,NCR,10.0,12.0,3.0,13.5
1,CAR,6.0,14.8,2.0,6.5
2,REGION I,3.8,10.5,1.3,4.8
3,REGION II,3.2,11.4,1.3,5.1
4,REGION III,3.4,7.2,1.1,5.0
5,REGION IVA,2.7,6.3,0.6,5.3
6,REGION IVB,1.8,5.5,0.5,1.0
7,REGION V,2.4,7.3,0.9,3.4
8,REGION VI,2.9,6.8,1.0,5.9
9,REGION VII,2.9,9.9,1.2,6.6


In [461]:
mortality = pd.read_csv('./region/mortality.txt', delimiter='\t',usecols=['AREA','NUMBER','PERCENT'])
mortality

,AREA,NUMBER,PERCENT
0,NCR,"71,050",13.4
1,CAR,"7,349",1.4
2,REGION I,"32,983",6.2
3,REGION II,"19,374",3.6
4,REGION III,"60,409",11.4
5,REGION IVA,"75,743",14.3
6,REGION IVB,"13,907",2.6
7,REGION V,"33,751",6.4
8,REGION VI,"48,990",9.2
9,REGION VII,"43,500",8.2


In [5]:
tablep = camelot.read_pdf('./region/popdensity.pdf', pages='23-74', flavor='stream')

In [8]:
popd = pd.DataFrame()

for i in range(52):
    popd = pd.concat([popd, tablep[i].df])

In [9]:
popd.replace('', np.NaN, inplace=True)
popd.fillna(method='ffill', inplace=True)
popd.drop(columns=[1,2,3,5,7,8,9,10], inplace=True)
popd.reset_index(drop=True, inplace=True)
popd.rename({0:'AREA', 4:'Land Area', 6:'Pop Density'}, axis=1,inplace=True)
popd = popd.replace(',','', regex=True)
ul = popd.loc[popd['Land Area'].str.contains('^(?!\D)\d*',regex=True)==False].index.tolist()
popd.drop(index=ul, inplace=True)
popd.reset_index(drop=True, inplace=True)
ulul = popd[(popd['AREA'].str.contains('Region') == False )|(popd['AREA'].str.contains('Table') == True)].index.tolist()
popd.drop(index=ulul, inplace=True)
popd.dropna(inplace=True)
popd.reset_index(inplace=True, drop=True)
popd.drop(index=10, inplace=True)
popd.at[0,'AREA'] = 'NCR'
popd.at[1,'AREA'] = 'CAR'
popd.at[5,'AREA'] = 'Region IVA'
popd.at[6,'AREA'] = 'Region IVB'
popd.at[16,'AREA'] = 'ARMM'
popd['AREA'] = popd['AREA'].str.partition(' – ')[0].str.upper()
popd

,AREA,Land Area,Pop Density
0,NCR,619.54,19137
1,CAR,19818.12,82
2,REGION I,12964.62,366
3,REGION II,29836.88,108
4,REGION III,21906.19,463
5,REGION IVA,16576.26,761
6,REGION IVB,29606.25,93
7,REGION V,18114.47,299
8,REGION VI,12773.46,328
9,REGION VII,10452.01,528


In [13]:
regions = pd.concat([hc[['AREA','Doctors','Nurses','Beds']], popd[['Pop Density', 'Land Area']], mortality['NUMBER'],final_host['Counts']],axis=1)
regions.set_index('AREA',drop=True, inplace=True)
regions.rename({'NUMBER':'Mortality', 'Counts':'Host_Count'}, axis=1, inplace=True)
regions = regions.replace(',','', regex=True)
regions = regions.astype(float)
regions.to_csv('Mapped Data.csv')
regions

,Doctors,Nurses,Beds,Mortality,Host_Count,Pop Density,Land Area
AREA,,,,,,,
NCR,10.0,12.0,13.5,71050.0,30.0,19137.0,619.54
CAR,6.0,14.8,6.5,7349.0,6.0,82.0,19818.12
REGION I,3.8,10.5,4.8,32983.0,2.0,366.0,12964.62
REGION II,3.2,11.4,5.1,19374.0,8.0,108.0,29836.88
REGION III,3.4,7.2,5.0,60409.0,2.0,463.0,21906.19
REGION IVA,2.7,6.3,5.3,75743.0,20.0,761.0,16576.26
REGION IVB,1.8,5.5,1.0,13907.0,28.0,93.0,29606.25
REGION V,2.4,7.3,3.4,33751.0,98.0,299.0,18114.47
REGION VI,2.9,6.8,5.9,48990.0,94.0,328.0,12773.46
